# load_model_testing

In [1]:
## Library
import pandas as pd
import numpy as np
import collections
import tensorflow as tf
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import MeanAbsoluteError, RootMeanSquaredError, MeanSquaredError
import sklearn
from matplotlib import pyplot as plt
from typing import Dict, Text

2023-05-30 15:22:57.859130: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-30 15:22:57.978885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 15:22:59.310277: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data initialize

In [2]:
movies_path = '../dataset/movies_metadata.csv'
rating_path = '../dataset/ratings_small.csv'

df_movies = pd.read_csv(movies_path).drop([19730, 29503, 35587])
df_rating = pd.read_csv(rating_path)

df_movies_cleaned = df_movies[['id', 'original_title']]
df_rating_cleaned = df_rating[['movieId', 'userId', 'rating']]
display(df_movies_cleaned.head(5))
display(df_rating_cleaned.head(5))
df_movies_cleaned['id'] = df_movies_cleaned['id'].astype('int64')

df_movies_cleaned.loc[:, 'id'] = pd.to_numeric(df_movies_cleaned['id'], errors='coerce').astype('Int64')
df_movies_cleaned = df_movies_cleaned.dropna(subset=['id']) # remove any line with NaN
display(df_movies_cleaned.head(5))

merged_dataset = pd.merge(df_rating_cleaned, df_movies_cleaned[['id', 'original_title']], left_on='movieId', right_on='id', how='left')

merged_dataset = merged_dataset[~merged_dataset['id'].isna()]
merged_dataset.dropna(inplace=True)
merged_dataset.drop('movieId', axis=1, inplace=True)
display(merged_dataset['id'].describe())
dataset_size = merged_dataset.shape[0] # count final size for dataset

merged_dataset.reset_index(drop=True, inplace=True)

display(merged_dataset.head())

df_movies_cleaned = df_movies_cleaned[~df_movies_cleaned['original_title'].duplicated()]
df_movies_cleaned['original_title'].head()

merged_dataset['userId'] = merged_dataset['userId'].astype(str)
# df_movies_cleaned['original_title'] = df_movies_cleaned['original_title'].astype(str)

rating_dict = dict(merged_dataset[['userId', 'original_title', 'rating']])
movies_dict = dict(df_movies_cleaned[['original_title']])

# Transfer to tf.tensor_slice
ratings_tf = tf.data.Dataset.from_tensor_slices(rating_dict)
movies_tf = tf.data.Dataset.from_tensor_slices(movies_dict)

movies_tf = movies_tf.map(lambda x: x["original_title"])

ratings_tf = ratings_tf.map(lambda x: {
    "original_title": x["original_title"],
    "rating": float(x["rating"]),
    "userId": x["userId"]
})
def slice_df_data(data_tf, total_size, test_rate=0.2):
    test_size = int(total_size * test_rate)
    train_size = total_size - test_size
    return data_tf.take(train_size), data_tf.skip(train_size).take(test_size)

train_ds, test_ds = slice_df_data(ratings_tf, dataset_size)
usrID_lookup = np.unique(np.concatenate([merged_dataset['userId'].to_numpy()]))
title_lookup = np.unique(np.concatenate([df_movies_cleaned['original_title']]))

/tmp/ipykernel_192963/1630367309.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(movies_path).drop([19730, 29503, 35587])


,id,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


,movieId,userId,rating
0,31,1,2.5
1,1029,1,3.0
2,1061,1,3.0
3,1129,1,2.0
4,1172,1,4.0


/tmp/ipykernel_192963/1630367309.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_cleaned['id'] = df_movies_cleaned['id'].astype('int64')
/tmp/ipykernel_192963/1630367309.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies_cleaned.loc[:, 'id'] = pd.to_numeric(df_movies_cleaned['id'], errors='coerce').astype('Int64')
/tmp/ipykernel_192963/1630367309.py:13: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setti

,id,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


count         44994.0
mean      6010.754901
std      15918.243916
min               2.0
25%             500.0
50%            1394.0
75%            2805.0
max          160718.0
Name: id, dtype: Float64

,userId,rating,id,original_title
0,1,2.5,1371,Rocky III
1,1,1.0,1405,Greed
2,1,4.0,2105,American Pie
3,1,2.0,2193,My Tutor
4,1,2.0,2294,Jay and Silent Bob Strike Back


2023-05-30 15:23:02.970141: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-30 15:23:03.164730: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-30 15:23:03.164809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-30 15:23:03.169149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-30 15:23:03.169223: I tensorflow/compile

## Model definition

In [6]:
# customized model for collaborative-based system
class CollabModel(tfrs.models.Model):
  def __init__(self, rating_w, retrieval_w, embedding_dim=64, L1_num=256, L2_num=128, L3_num=32, act_func="relu") -> None:
    super().__init__()
    embedding_dim = embedding_dim # number of dimension for embedding

    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=title_lookup, mask_token=None),
      tf.keras.layers.Embedding(len(title_lookup) + 1, embedding_dim)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=usrID_lookup, mask_token=None),
      tf.keras.layers.Embedding(len(usrID_lookup) + 1, embedding_dim)
    ])
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(L1_num, activation=act_func), # first layer
        tf.keras.layers.Dense(L2_num, activation=act_func), # second layer
        tf.keras.layers.Dense(L3_num, activation=act_func),# third layer
        tf.keras.layers.Dense(1), # output layer
    ])

    self.rating_eval: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss = MeanSquaredError(),
        metrics = [RootMeanSquaredError(), MeanAbsoluteError()],
    )
    self.retrieval_eval: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies_tf.batch(128).map(self.movie_model)
        )
    )

    self.rating_weight = rating_w
    self.retrieval_weight = retrieval_w

  # overloading call function
  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    user_embeddings = self.user_model(features["userId"])
    movie_embeddings = self.movie_model(features["original_title"])
    
    return (
        user_embeddings,
        movie_embeddings,
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  # overloading compute_loss function
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor: 
    label_r = features.pop("rating")
    user_embeddings, movie_embeddings, rating_pred = self(features)
    rating_loss = self.rating_eval(labels=label_r, predictions=rating_pred)
    retrieval_loss = self.retrieval_eval(user_embeddings, movie_embeddings)

    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

## Load model data

In [8]:
load_path = '../model/MRS_v2_e64_12.h5'

model = CollabModel(rating_w=1.0, retrieval_w=1.0, embedding_dim=64, L1_num=256, L2_num=128, L3_num=32, act_func="relu")
model.built = True
model.load_weights(load_path)

model.summary()

ValueError: Weights for model 'sequential_10' have not yet been created. Weights are created when the model is first called on inputs or `build()` is called with an `input_shape`.